In [1]:
import requests
from bs4 import BeautifulSoup

# 네이버 뉴스에서 '부동산' 키워드로 검색한 URL
url = "https://search.naver.com/search.naver?where=news&query=부동산"

# 요청 보내기
headers = {"User-Agent": "Mozilla/5.0"}  # User-Agent 설정 (크롤링 차단 방지)
response = requests.get(url, headers=headers)
response.raise_for_status()

# HTML 파싱
soup = BeautifulSoup(response.text, "html.parser")

# 뉴스 기사 제목과 링크 찾기
news_items = soup.select(".news_tit")  # 네이버 뉴스 검색 결과의 제목 클래스

# 결과 출력
for idx, news in enumerate(news_items, 1):
    title = news.get_text()
    link = news["href"]
    print(f"{idx}. {title}")
    print(f"   링크: {link}")


1. 현대차증권 VIP 자산관리 강화…미국 부동산 투자자문 MOU
   링크: https://www.yna.co.kr/view/AKR20250305051000008?input=1195m
2. [단독] '금호석유화학 본사' 을지로 시그니쳐타워 매물로 나왔다
   링크: https://www.hankyung.com/article/202503053993r
3. '북항 재개발 특혜 의혹' 부동산개발업체 대표, 구속기소
   링크: https://www.newsis.com/view/NISX20250305_0003086948
4. 공정위, 롯데 계열사 현장 조사⋯부동산 고가 거래 혐의
   링크: http://www.inews24.com/view/1819769
5. 2030세대가 부동산 ‘영끌’ 하는 데는 ‘이 경험’도 큽니다
   링크: https://www.khan.co.kr/article/202503041551001
6. 서울 상업용 부동산 거래 '반토막’
   링크: http://www.fntimes.com/html/view.php?ud=202503050947416453e41d7fc6c2_18
7. 5조 불어난 가계대출에 깜짝 놀란 정부…부동산 점검회의 연다
   링크: https://www.news1.kr/economy/trend/5708353
8. [홈플러스 법정관리] 1조2000억 내준 메리츠금융, 부동산PF 트리거 '촉각...
   링크: https://www.bloter.net/news/articleView.html?idxno=632597
9. 지방소멸 위험 저평가 했다가... 홈플러스에 물린 부동산 펀드
   링크: https://biz.chosun.com/distribution/food/2025/03/05/RA5AM62DBVHDHKP5WROXO2H324/?utm_source=naver&utm_medium=original&utm_campaign=biz
10. 홈플러스 기업회생절차 개시 후폭풍…상업용 부동산 시장 격랑 휘말리나...
   링크: https

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 1️⃣ Chrome WebDriver 설정 (헤드리스 모드 해제)
options = Options()
# options.add_argument("--headless")  # ❌ 헤드리스 모드 제거 (창이 보이도록)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# 최신 크롬 드라이버 자동 다운로드 및 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 2️⃣ 네이버 뉴스 검색 페이지 열기
search_url = "https://search.naver.com/search.naver?where=news&query=부동산"
driver.get(search_url)

# 페이지 로딩 대기
wait = WebDriverWait(driver, 10)

# 3️⃣ 뉴스 제목, 링크, 언론사 크롤링
news_list = []

try:
    # 뉴스 목록 로딩 대기
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".news_tit")))

    news_items = driver.find_elements(By.CSS_SELECTOR, ".news_tit")
    press_items = driver.find_elements(By.CSS_SELECTOR, ".info.press")

    for news, press in zip(news_items, press_items):
        title = news.text  # 뉴스 제목
        link = news.get_attribute("href")  # 뉴스 링크
        press_name = press.text  # 언론사 이름

        # 4️⃣ 새 창에서 뉴스 본문 가져오기
        driver.execute_script(f"window.open('{link}', '_blank');")  # 새 창 열기
        driver.switch_to.window(driver.window_handles[-1])  # 새 창으로 전환
        
        try:
            time.sleep(2)  # 페이지 로딩 대기
            
            # 5️⃣ 사이트별 본문 크롤링 로직 추가
            if "news.naver.com" in link:
                content_elements = WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#dic_area p"))
                )
            elif "yna.co.kr" in link:  # 연합뉴스
                content_elements = WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".story-news p"))
                )
            else:
                content_elements = []

            # 본문 가져오기
            content = "\n".join([p.text for p in content_elements if p.text.strip()])
            if not content:
                content = "본문을 가져올 수 없습니다."

        except:
            content = "본문을 가져올 수 없습니다."  # 본문이 없거나 로딩 실패 시

        # 데이터 저장
        news_list.append({
            "제목": title,
            "언론사": press_name,
            "링크": link,
            "본문": content
        })

        # 현재 창 닫고, 원래 창으로 돌아가기
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

except Exception as e:
    print(f"오류 발생: {e}")

# 크롬 드라이버 종료
driver.quit()

# 5️⃣ 크롤링한 데이터 DataFrame으로 변환
df = pd.DataFrame(news_list)



# 7️⃣ CSV 파일로 저장 (원할 경우)
df.to_csv("real_estate_news.csv", index=False, encoding="utf-8-sig")
